## **Chapter V. Task**

We will continue our training with a problem from Kaggle.com. 
In this chapter, we will implement all the validation schemes, some hyperparameter tuning methods, and feature selection methods described above. Measure quality metrics on training and test samples. Will detect overfitted models and regularize them. And dive deeper with native model estimation and comparison.

### 1. **Answer the questions from the introduction**

#### a. What is leave-one-out? Provide limitations and strengths.  

Leave-One-Out кросс валидация - это частный случай k-fold Cross Validation, где k = n. Данные размерности 10 делятся на обучающую и тестовую выборки по принципу 9 обучающей и 1 тестовая на каждой итерации для всех данных. При этом,каждый набор данных __ровно один раз__ становится тестовым (см. пример кода.). Другими словами, для **_n_** образцов мы имеем **_n-1_** тренировочных данных и **1** тестовый набор.
Достоинства:
 * максимальное использование данных для обучения
 * не зависит от случайного разбиения
 * эффективен только на маленьких наборах данных

Недостатки:
 * высокая стоимость вычислительных операций для больших выборок данных - O(n)
 * высокая дисперсия оценки
 * может недооценивать ошибку на новых данных

```Python
>>> from sklearn.model_selection import LeaveOneOut

>>> X = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
>>> loo = LeaveOneOut()
>>> for train, test in loo.split(X):
...     print("%s %s" % (train, test))

[1 2 3 4 5 6 7 8 9] [0]
[0 2 3 4 5 6 7 8 9] [1]
[0 1 3 4 5 6 7 8 9] [2]
[0 1 2 4 5 6 7 8 9] [3]
[0 1 2 3 5 6 7 8 9] [4]
[0 1 2 3 4 6 7 8 9] [5]
[0 1 2 3 4 5 7 8 9] [6]
[0 1 2 3 4 5 6 8 9] [7]
[0 1 2 3 4 5 6 7 9] [8]
[0 1 2 3 4 5 6 7 8] [9]
```

Источник: https://scikit-learn.org/stable/modules/cross_validation.html#leave-one-out

#### b. How do Grid Search, Randomized Grid Search, and Bayesian optimization work?  

In [1]:
A = {1, 2, 3, 4, 5}
B = {6, 7, 8, 9}
C = {10, 11, 12}

def cartesian_product(*sets):
    """
    Декартово произведение множеств
    """
    result = [[]]
    for pool in sets:
        result = [x + [y] for x in result for y in pool]
    return [tuple(item) for item in result]

decart = cartesian_product(A, B, C)

> Этапы работы поиска наилучших гиперпараметров через Grid Search:
 1. **Определение пространства внешних гиперпараметров**. На первом этапе необходимо обозначить какие гиперпараметры мы ищем. Параметр должен содержать множество значений, по которым мы будем проводить поиск. Параметров может быть несколько. Несколько параметров собираются в единый словарь, где ключи — имена гиперпараметров, а значения — списки возможных значений для каждого параметра.
 2. **Создание сетки гиперпараметров**. Создается декартово произведение всех списков значений, получая полный набор всех возможных комбинаций гиперпараметров.
 3. **Проход по списку гиперпараметров**. Алгоритм GridSearch последовательно проходит по каждой комбинации гиперпараметров, обучается и оценивается с помощью k-fold CV, вычисляется средняя оценка гиперпараметров по всем фолдам для заданной метрики.
 4. **Сохранение и возвращение результата**. По окончании подбора параметров выбирается комбинация с наилучшей средней оценкой. GridSearch возвращает: лучшие гиперпараметры `best_params_`; лучшие оценки `best_score_`; полную информацию по всем комбинациям `cv_results_`.

> Randomized Grid Search:
 * Принцип работы Randomized Grid Search схож с работой Grid Search, но не создает полную сетку всех комбинаций гиперпараметров. Вместо этого поиск происходит случайным образом. Еще одно отличие - это наличие итераций. На каждой итерации `n_iter` случайным образом генерируется комбинация гиперпараметров, происходит обучение модели и оценка её качества с помощью k-fold CV на всех фолдах. Случайный выбор позволяет ускорить процесс подбора гиперпараметров. В таком случае, гиперпараметры следует задавать не только дискретным образом, но и в диапазоне и\или распределении, например: `stats.uniform(0.1, 10)`, здесь будет создано распределение случайных чисел от 0.1 до 10 + 0.1. Случайное число из данного распределения будет получено внутри GridSearch.

> Bayesian optimization


#### c. Explain classification of feature selection methods. Explain how Pearson and Chi2 work. Explain how Lasso works. Explain what permutation significance is. Become familiar with SHAP.  

### **2. Introduction — do all the preprocessing from the previous lesson**

In [2]:
import pandas as pd
import numpy as np
import re
from math import ceil

#### a. Read all the data.

In [3]:
df = pd.read_json("../datasets/train.json", convert_dates=["created"]).reset_index(drop=True)
    # pd.read_json("../datasets/test.json") - здесь нет признака 'interest_level'

#### b. Preprocess the "Interest Level" feature.

In [4]:
df["interest_level"] = df["interest_level"].map({"low": 0, "medium": 1, "high": 2})

#### c. Create features: 'Elevator', 'HardwoodFloors', 'CatsAllowed', 'DogsAllowed', 'Doorman', 'Dishwasher', 'NoFee', 'LaundryinBuilding', 'FitnessCenter', 'Pre-War', 'LaundryinUnit', 'RoofDeck', 'OutdoorSpace', 'DiningRoom', 'HighSpeedInternet', 'Balcony', 'SwimmingPool', 'LaundryInBuilding', 'NewConstruction', 'Terrace'.  

In [5]:
pattern = re.compile(r"[\[\]\'\"\s]")
remove_sym = (lambda x: list([re.sub(pattern, '', elem) for elem in x]))
df["features"] = df["features"].apply(remove_sym)

target = ["price"]
rooms = ["bedrooms", "bathrooms"]
features = ['Elevator', 'HardwoodFloors', 'CatsAllowed', 'DogsAllowed',
            'Doorman', 'Dishwasher', 'NoFee', 'LaundryinBuilding',
            'FitnessCenter', 'Pre-War', 'LaundryinUnit', 'RoofDeck',
            'OutdoorSpace', 'DiningRoom', 'HighSpeedInternet', 'Balcony',
            'SwimmingPool', 'LaundryInBuilding', 'NewConstruction', 'Terrace',]
for feature in features:
    df[feature] = (
        df["features"]
        .apply(lambda x: 1 if feature in x else 0)
    )

### 3. **Implement the next methods:**

#### a. Split data into 2 parts randomly with parameter test_size (ratio from 0 to 1), return training and test samples.

In [6]:
# 3.a
def my_train_test_split(X, y, test_size=0.25):
    n = len(X)
    if isinstance(test_size, int):
        if test_size <= 0 or test_size >= n:
            raise ValueError("'test_size' must be between 1 and len(X)-1")
    elif isinstance(test_size, float):
        if test_size <= 0.0 or test_size >= 1.0:
            raise ValueError("'test_size' must be between 0.0 and 1.0")
        test_size = ceil(n * test_size)
    else:
        raise ValueError("'test_size' must be float or int")

    if n != len(y):
        raise ValueError("Length of 'X' and 'y' must be equal")

    idx = X.index.to_numpy()
    train_idxs = idx[test_size:]
    test_idxs = idx[:test_size]

    return (pd.concat([X.loc[train_idxs], y.loc[train_idxs]], axis=1),
            pd.concat([X.loc[test_idxs], y.loc[test_idxs]], axis=1))


train, test = my_train_test_split(
    df.drop(columns=target),
    df[target],
    test_size=0.2
)
display(
    train.shape[0] + test.shape[0] == len(df)
)

train_idx, test_idx = train.index, test.index

print(train_idx.intersection(test_idx).empty,
      test_idx.intersection(train_idx).empty)

True

True True


#### b. Randomly split data into 3 parts with parameters validation_size and test_size, return train, validation and test samples.

In [7]:
# 3.b
def my_train_validation_test_split(X, y, validation_size=0.25, test_size=0.25):
    n = len(X)
    if isinstance(test_size, int) and isinstance(validation_size, int):
        if test_size <= 0 or test_size >= n:
            raise ValueError("'test_size' must be between 1 and len(X)-1")
        if validation_size <= 0 or validation_size >= n:
            raise ValueError("'validation_size' must be between 1 and len(X)-1")
    elif isinstance(test_size, float) and isinstance(validation_size, float):
        if test_size <= 0.0 or test_size >= 1.0:
            raise ValueError("'test_size' must be between 0.0 and 1.0")
        if validation_size <= 0.0 or validation_size >= 1.0:
            raise ValueError("'validation_size' must be between 0.0 and 1.0")
        if test_size + validation_size >= 1:
            raise ValueError("test_size + validation_size must be < 1")
        test_size = ceil(n * test_size)
        validation_size = ceil(n * validation_size)
    else:
        raise ValueError("'test_size'/'validation_size' must be float or int")

    idx = X.index.to_numpy()

    if n != len(y) or not np.array_equal(idx, y.index.to_numpy()):
        raise ValueError("Length of 'X' and 'y' must be equal")
    if test_size + validation_size >= n:
        raise ValueError("test_size + validation_size must be < len(X)")

    train_idxs = idx[test_size + validation_size:]
    test_idxs = idx[:test_size]
    validation_idxs = idx[test_size:test_size + validation_size]

    return (pd.concat([X.loc[train_idxs], y.loc[train_idxs]], axis=1),
            pd.concat([X.loc[validation_idxs], y.loc[validation_idxs]], axis=1),
            pd.concat([X.loc[test_idxs], y.loc[test_idxs]], axis=1))


train, validation, test = my_train_validation_test_split(
    df.drop(columns=target),
    df[target],
    test_size=0.2,
    validation_size=0.2
)

display(train.shape[0] + validation.shape[0] + test.shape[0] == len(df))

train_idx = train.index
val_idx = validation.index
test_idx = test.index

print(train_idx.intersection(val_idx).empty,
      train_idx.intersection(test_idx).empty,
      val_idx.intersection(test_idx).empty)
# train, validation = my_train_test_split(
#     train.drop(columns=target),
#     train[target],
#     test_size=0.25  # 0.2 / (1 - 0.2) = 0.25
# )

True

True True True


#### c. Split data into 2 parts with parameter date_split, return train and test samples split by date_split param.

In [ ]:
def my_train_test_split(X, y, *, test_size=0.25, random_state=None, shuffle=True):
    random = np.random.RandomState(random_state)
    idx = X.index.to_numpy()

    # Проверка типов test_size и преобразование в количество объектов
    if isinstance(test_size, float):
        test_size = ceil(len(X) * test_size)
    elif isinstance(test_size, int):
        pass
    else:
        raise ValueError("test_size must be float or int")

    # Флаг shuffle отвечает за перемешивание выборки перед разбиением
    if shuffle:
        idx = random.permutation(idx)

    test_idxs = idx[:test_size]
    train_idxs = idx[test_size:]

    return (X.loc[train_idxs], X.loc[test_idxs], y.loc[train_idxs], y.loc[test_idxs])


# my_train_test_split(df.drop(columns=target), df[target], test_size=0.2, random_state=21)
my_train_X, my_test_X, my_train_y, my_test_y = my_train_test_split(df.drop(columns=target), df[target], test_size=4, random_state=21)

train_X, test_X, train_y, test_y = train_test_split(df.drop(columns=target), df[target], test_size=4, random_state=21)
display(
    train_X.equals(my_train_X),
    train_y.equals(my_train_y),
    test_X.equals(my_test_X),
    test_y.equals(my_test_y),
) # True

#### d. Split data into 3 parts with parameters validation_date and test_date, return train, validation and test samples split by input params.

In [13]:
df["year"] = df["created"].dt.year
df["month"] = df["created"].dt.month

In [14]:
df["month"].value_counts()

month
6    17144
4    16411
5    15797
Name: count, dtype: int64

#### e. Make split procedure determenistic. What does it mean?

### 4. **Implement the next cross-validation methods:**

#### a. K-Fold, where k is the input parameter, returns a list of train and test indices.

#### b. Grouped K-Fold, where k and group_field are input parameters, returns list of train and test indices.

#### c. Stratified K-fold, where k and stratify_field are input parameters, returns list of train and test indices.

#### d. Time series split, where k and date_field are input parameters, returns list of train and test indices.

### 5. **Cross-validation comparison**

#### a. Apply all the validation methods implemented above to our dataset. To apply Stratified algorithm you should preprocess target.

#### b. Apply the appropriate methods from sklearn.

#### c. Compare the resulting feature distributions for the training part of the dataset between sklearn and your implementation.

#### d. Compare all validation schemes. Choose the best one. Explain your choice.

### 6. **Feature Selection**

#### a. Fit a Lasso regression model with normalized features. Use your method for splitting samples into 3 parts by field created with 60/20/20 ratio — train/validation/test.

#### b. Sort features by weight coefficients from model, fit model to top 10 features and compare quality.

#### c. Implement method for simple feature selection by nan-ratio in feature and correlation. Apply this method to feature set and take top 10 features, refit model and measure quality.

#### d. Implement permutation importance method and take top 10 features, refit model and measure quality.

#### e. Import Shap and also refit model on top 10 features.

#### f. Compare the quality of these methods for different aspects — speed, metrics and stability.

### 7. **Hyperparameter optimization**

#### a. Implement grid search and random search methods for alpha and l1_ratio for sklearn's ElasticNet model.

#### b. Find the best combination of model hyperparameters.

#### c. Fit the resulting model.

#### d. Import optuna and configure the same experiment with ElasticNet.

#### e. Estimate metrics and compare approaches.

f. Run optuna on one of the cross-validation schemes.